<a href="https://colab.research.google.com/github/satojkovic/ToT-Colab/blob/main/tree_of_thoughts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree of Thoughts (ToT) Demo

このノートブックはTree of Thoughtsアルゴリズムの実装とデモンストレーションです。  
論文: [Tree of Thoughts: Deliberate Problem Solving with Large Language Models](https://arxiv.org/abs/2305.10601)

## アルゴリズムの概要

Tree of Thoughtsは従来の単発推論とは異なり、**思考の木構造**を構築して問題を解決します：

1. **思考生成**: 各ステップで複数の思考候補を生成
2. **状態評価**: 各思考状態の価値を評価
3. **選択**: 最も有望な思考を選択して次のステップへ
4. **探索**: 幅優先探索で最適解を体系的に探索

## 1. 環境設定とライブラリのインストール

In [ ]:
# 必要なライブラリのインストール
!pip install openai sympy pandas numpy matplotlib seaborn
!pip install tree-of-thoughts-llm

In [ ]:
# オフィシャルコードのクローン（代替案）
# !git clone https://github.com/princeton-nlp/tree-of-thought-llm.git
# %cd tree-of-thought-llm
# !pip install -e .

In [ ]:
import os
import json
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any

# OpenAI API設定
import openai
from google.colab import userdata

# OpenAI API キーの設定（Google Colab Secrets使用）
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## 2. Tree of Thoughtsの基本的な使用例

In [ ]:
from tot.methods.bfs import solve
from tot.tasks.game24 import Game24Task

# パラメータ設定
args = argparse.Namespace(
    backend='gpt-4',
    temperature=0.7,
    task='game24',
    naive_run=False,
    prompt_sample=None,
    method_generate='propose',
    method_evaluate='value',
    method_select='greedy',
    n_generate_sample=1,
    n_evaluate_sample=3,
    n_select_sample=5
)

print("Tree of Thoughts設定:")
print(f"- モデル: {args.backend}")
print(f"- 思考生成: {args.method_generate}")
print(f"- 状態評価: {args.method_evaluate}")
print(f"- 選択方法: {args.method_select}")
print(f"- 選択候補数: {args.n_select_sample}")

## 3. Game24タスクのデモンストレーション

Game24は4つの数字を使って24を作る数学パズルです。

In [ ]:
# Game24タスクの初期化
task = Game24Task()

# タスクの詳細表示
print(f"データセットサイズ: {len(task)}")
print(f"問題の例: {task.get_input(0)}")
print(f"探索ステップ数: {task.steps}")

# 最初の5つの問題を表示
print("\n問題例:")
for i in range(5):
    print(f"問題{i+1}: {task.get_input(i)}")

In [ ]:
# 単一問題の解決
problem_idx = 0  # 解決したい問題のインデックス
input_numbers = task.get_input(problem_idx)

print(f"問題: {input_numbers}")
print("解決中...\n")

# ToTアルゴリズムで解決
solutions, info = solve(args, task, problem_idx)

print("\n=== 解決結果 ===")
for i, solution in enumerate(solutions):
    print(f"解法{i+1}:")
    print(solution)
    print(f"正解: {task.test_output(problem_idx, solution)}")
    print()

## 4. アルゴリズムの動作可視化

In [ ]:
def visualize_search_process(info: Dict[str, Any]):
    """
    探索プロセスを可視化
    """
    steps = info['steps']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Tree of Thoughts 探索プロセス', fontsize=16)
    
    # 1. 各ステップでの候補数
    step_nums = []
    candidate_counts = []
    selected_counts = []
    
    for step_info in steps:
        step_nums.append(step_info['step'])
        candidate_counts.append(len(step_info['new_ys']))
        selected_counts.append(len(step_info['select_new_ys']))
    
    axes[0, 0].bar(step_nums, candidate_counts, alpha=0.7, label='生成候補数')
    axes[0, 0].bar(step_nums, selected_counts, alpha=0.7, label='選択候補数')
    axes[0, 0].set_xlabel('ステップ')
    axes[0, 0].set_ylabel('候補数')
    axes[0, 0].set_title('各ステップでの候補数')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 評価値の分布
    all_values = []
    for step_info in steps:
        all_values.extend(step_info['values'])
    
    axes[0, 1].hist(all_values, bins=20, alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('評価値')
    axes[0, 1].set_ylabel('頻度')
    axes[0, 1].set_title('評価値の分布')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 各ステップでの最高評価値
    max_values = []
    avg_values = []
    
    for step_info in steps:
        values = step_info['values']
        max_values.append(max(values) if values else 0)
        avg_values.append(np.mean(values) if values else 0)
    
    axes[1, 0].plot(step_nums, max_values, marker='o', label='最高評価値')
    axes[1, 0].plot(step_nums, avg_values, marker='s', label='平均評価値')
    axes[1, 0].set_xlabel('ステップ')
    axes[1, 0].set_ylabel('評価値')
    axes[1, 0].set_title('ステップごとの評価値推移')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 選択された候補の評価値
    selected_values = []
    for step_info in steps:
        values = step_info['values']
        select_count = len(step_info['select_new_ys'])
        if values:
            sorted_values = sorted(values, reverse=True)
            selected_values.append(sorted_values[:select_count])
    
    if selected_values:
        for i, step_values in enumerate(selected_values):
            axes[1, 1].scatter([i] * len(step_values), step_values, alpha=0.7)
    
    axes[1, 1].set_xlabel('ステップ')
    axes[1, 1].set_ylabel('評価値')
    axes[1, 1].set_title('選択された候補の評価値')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 探索プロセスの可視化
if 'info' in locals():
    visualize_search_process(info)

## 5. 複数問題での性能評価

In [ ]:
def evaluate_multiple_problems(task, args, start_idx=0, end_idx=10):
    """
    複数の問題でToTの性能を評価
    """
    results = []
    
    for i in range(start_idx, min(end_idx, len(task))):
        print(f"問題 {i+1}/{end_idx}: {task.get_input(i)}")
        
        try:
            solutions, info = solve(args, task, i, to_print=False)
            
            # 最良の解を評価
            best_solution = solutions[0] if solutions else ""
            test_result = task.test_output(i, best_solution)
            
            results.append({
                'problem_idx': i,
                'input': task.get_input(i),
                'solution': best_solution,
                'correct': test_result['r'],
                'steps': len(info['steps']) if 'steps' in info else 0
            })
            
            print(f"結果: {'正解' if test_result['r'] else '不正解'}")
            print()
            
        except Exception as e:
            print(f"エラー: {e}")
            results.append({
                'problem_idx': i,
                'input': task.get_input(i),
                'solution': '',
                'correct': 0,
                'steps': 0
            })
    
    return results

# 複数問題での評価実行
print("複数問題での性能評価を開始...")
evaluation_results = evaluate_multiple_problems(task, args, start_idx=0, end_idx=5)

# 結果の分析
correct_count = sum(1 for r in evaluation_results if r['correct'])
total_count = len(evaluation_results)
accuracy = correct_count / total_count if total_count > 0 else 0

print(f"\n=== 評価結果 ===")
print(f"正解数: {correct_count}/{total_count}")
print(f"精度: {accuracy:.2%}")

# 結果をDataFrameで表示
df_results = pd.DataFrame(evaluation_results)
print("\n詳細結果:")
print(df_results[['problem_idx', 'input', 'correct']].to_string(index=False))

## 6. 異なるアルゴリズム設定の比較

In [ ]:
def compare_configurations():
    """
    異なるToT設定の比較
    """
    configurations = [
        {
            'name': 'ToT (greedy)',
            'method_generate': 'propose',
            'method_evaluate': 'value',
            'method_select': 'greedy',
            'n_select_sample': 3
        },
        {
            'name': 'ToT (sample)',
            'method_generate': 'propose',
            'method_evaluate': 'value',
            'method_select': 'sample',
            'n_select_sample': 3
        },
        {
            'name': 'ToT (wide search)',
            'method_generate': 'propose',
            'method_evaluate': 'value',
            'method_select': 'greedy',
            'n_select_sample': 5
        }
    ]
    
    comparison_results = []
    
    for config in configurations:
        print(f"\n設定: {config['name']}")
        
        # 設定を更新
        test_args = argparse.Namespace(**vars(args))
        for key, value in config.items():
            if key != 'name':
                setattr(test_args, key, value)
        
        # 小さなサンプルでテスト
        results = evaluate_multiple_problems(task, test_args, start_idx=0, end_idx=3)
        
        accuracy = sum(1 for r in results if r['correct']) / len(results)
        comparison_results.append({
            'configuration': config['name'],
            'accuracy': accuracy,
            'correct_count': sum(1 for r in results if r['correct']),
            'total_count': len(results)
        })
        
        print(f"精度: {accuracy:.2%}")
    
    # 結果の可視化
    df_comparison = pd.DataFrame(comparison_results)
    
    plt.figure(figsize=(10, 6))
    plt.bar(df_comparison['configuration'], df_comparison['accuracy'])
    plt.title('異なるToT設定の性能比較')
    plt.xlabel('設定')
    plt.ylabel('精度')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return comparison_results

# 設定比較の実行
print("異なるアルゴリズム設定の比較を開始...")
comparison_results = compare_configurations()

print("\n=== 比較結果 ===")
for result in comparison_results:
    print(f"{result['configuration']}: {result['accuracy']:.2%} ({result['correct_count']}/{result['total_count']})")

## 7. カスタム問題の作成と解決

In [ ]:
def solve_custom_problem(numbers_str: str):
    """
    カスタム問題を解決
    """
    print(f"カスタム問題: {numbers_str}")
    
    # 一時的なタスククラスを作成
    class CustomGame24Task(Game24Task):
        def __init__(self, custom_input):
            super().__init__()
            self.custom_input = custom_input
        
        def get_input(self, idx):
            return self.custom_input
    
    custom_task = CustomGame24Task(numbers_str)
    
    try:
        solutions, info = solve(args, custom_task, 0)
        
        print("\n=== 解決結果 ===")
        for i, solution in enumerate(solutions):
            print(f"解法{i+1}:")
            print(solution)
            result = custom_task.test_output(0, solution)
            print(f"正解: {result['r'] == 1}")
            print()
        
        return solutions, info
        
    except Exception as e:
        print(f"エラー: {e}")
        return [], {}

# カスタム問題の例
custom_problems = [
    "1 2 3 4",
    "4 1 8 7",
    "2 3 5 6"
]

print("カスタム問題の解決:")
for problem in custom_problems:
    print("\n" + "="*50)
    solve_custom_problem(problem)

## 8. まとめと今後の展開

### Tree of Thoughtsの特徴

1. **構造化された探索**: 単発推論ではなく、思考の木構造を構築
2. **中間状態の評価**: 各思考ステップで状態を評価し、最適な経路を選択
3. **柔軟な設定**: 生成・評価・選択の各段階で異なる戦略を選択可能
4. **高い解決能力**: 複雑な問題に対して体系的なアプローチを提供

### 応用可能な分野

- **数学的問題解決**: Game24のような数値計算問題
- **創作活動**: 小説や詩の執筆
- **論理パズル**: クロスワードパズルなど
- **戦略的思考**: ゲーム理論や意思決定問題

### 今後の改良点

1. **効率化**: 計算コストの削減とレスポンス時間の向上
2. **評価関数の改善**: より正確な中間状態評価
3. **動的な探索**: 問題の複雑さに応じた適応的な探索深度
4. **マルチモーダル対応**: テキスト以外の入力への対応